In [1]:
# Import Libraries 
# ----------------------------------------------
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Input Parameter
# ----------------------------------------------

# Discretization
c1=20           # Number of grid points per dominant wavelength
c2=0.5          # CFL-Number
nx=321          # number of grid points in x-direction
nz=180          # number of grid points in z-direction
Tt=2
 
# Source Signal
f0= 5      # Center frequency Ricker-wavelet
q0= 1      # Maximum amplitude Ricker-Wavelet
xscr = int(nx/2);  zscr = 25; # Source position (in grid points)

In [3]:
# Velocity and density
model_vp=3000*np.ones((nx,nz))   # P-wave velocity in m/s
model_vs=1500*np.ones((nx,nz)) # P-wave velocity in m/s
rho= 2200*np.ones((nx,nz))      # Density in g/cm^3

for j in range((nx//2)-1,nx):
    model_vp[j,:]=model_vp[j,:]*1.3
    
for j in range((nx//2)-1,nx):
    model_vs[j,:]=model_vs[j,:]*1.3
   
for j in range((nx//2)-1,nx):
    rho[j,:]=rho[j,:]*1.25

In [4]:
## Preparation 

# Init wavefields
vx=np.zeros((nx,nz))
vz=np.zeros((nx,nz))
txx=np.zeros((nx,nz))
tzz=np.zeros((nx,nz))
txz=np.zeros((nx,nz))

# Calculate Lame  Parameters
L=rho*(model_vp**2 - 2*model_vs**2)
M=rho*model_vs**2 

cmin=model_vp[:].min()  # Lowest P-wave velocity
cmax=model_vp[:].max()  # Highest P-wave velocity
fmax=2*f0              # Maximum frequency
dx=cmin/(fmax*c1)      # Spatial discretization (in m)
dz=dx 
dt=dx/(cmax)*c2        # Temporal discretization (in s)
lampda_min=cmin/fmax   # Smallest wavelength

# Create space and time vector
x=np.arange(0,dx*nx-dx,dx) # Space vector
z=np.arange(0,(dz*nz-dz),dz) # Space vector
t=np.arange(0,(Tt-dt),dt)    # Time vector
nt=len(t)       # Number of time steps

In [5]:
# Source signal - Ricker-wavelet
tau=np.pi*f0*(t-1.5/f0)
q=q0*(1-2*tau**2)*np.exp(-tau**2)

# Init Seismograms
#Seismogram=np.zeros((3,nt)) # 3 Seismogramme eingerichtet
seismo=np.zeros((nx,nt))
seismoy=np.zeros((nx,nt))
seismotzz=np.zeros((nx,nt))
# Calculation of some coefficients
c1=9/8
c2=1/24

In [ ]:
for n in range(1,nt):
#    4:nx-6=4:nx-6
#    4:nz-6=4:nz-6
    # Update velocity
    # Calculating spatial derivative
    txx_x = (c1*(txx[4:nx-6,4:nz-6]-txx[3:nx-7,4:nz-6])+c2*(txx[2:nx-8,4:nz-6]-txx[5:nx-5,4:nz-6]))/dx
    txz_x = (c1*(txz[5:nx-5,4:nz-6]-txz[4:nx-6,4:nz-6])+c2*(txz[3:nx-7,4:nz-6]-txz[6:nx-4,5:nz-5]))/dx
            
    tzz_z = (c1*(tzz[4:nx-6,5:nz-5]-tzz[4:nx-6,4:nz-6])+c2*(tzz[4:nx-6,3:nz-7]-tzz[4:nx-6,6:nz-4]))/dz
    txz_z = (c1*(txz[4:nx-6,4:nz-6]-txz[4:nx-6,3:nz-7])+c2*(txz[4:nx-6,2:nz-8]-txz[4:nx-6,5:nz-5]))/dz
           
    # Update velocity
    vx[4:nx-6,4:nz-6]=vx[4:nx-6,4:nz-6]+(dt/rho[4:nx-6,4:nz-6])*(txx_x+txz_z)
    vz[4:nx-6,4:nz-6]=vz[4:nx-6,4:nz-6]+(dt/rho[4:nx-6,4:nz-6])*(txz_x+tzz_z)

    # Inject source wavelet
    txx[xscr,zscr]=txx[xscr,zscr]+q[n]
    tzz[xscr,zscr]=tzz[xscr,zscr]+q[n]

    # Calculating spatial derivative

    vx_x = (c1*(vx[5:nx-5,4:nz-6]-vx[4:nx-6,4:nz-6])+c2*(vx[3:nx-7,4:nz-6]-vx[6:nx-4,4:nz-6]))/dx
    vz_x = (c1*(vz[4:nx-6,4:nz-6]-vz[3:nx-7,4:nz-6])+c2*(vz[2:nx-8,4:nz-6]-vz[5:nx-5,4:nz-6]))/dx        

    vz_z = (c1*(vz[4:nx-6,4:nz-6]-vz[4:nx-6,3:nz-7])+c2*(vz[4:nx-6,2:nz-8]-vz[4:nx-6,5:nz-5]))/dz
    vx_z = (c1*(vx[4:nx-6,5:nz-5]-vx[4:nx-6,4:nz-6])+c2*(vx[4:nx-6,3:nz-7]-vx[4:nx-6,6:nz-4]))/dz
                       
    # Update pressure

    txx[4:nx-6,4:nz-6]=txx[4:nx-6,4:nz-6]+(L[4:nx-6,4:nz-6]+2*M[4:nx-6,4:nz-6])*dt*vx_x+L[4:nx-6,4:nz-6]*dt*vz_z
    tzz[4:nx-6,4:nz-6]=tzz[4:nx-6,4:nz-6]+(L[4:nx-6,4:nz-6]+2*M[4:nx-6,4:nz-6])*dt*vz_z+L[4:nx-6,4:nz-6]*dt*vx_x
    txz[4:nx-6,4:nz-6]=txz[4:nx-6,4:nz-6]+M[4:nx-6,4:nz-6]*dt*(vx_z+vz_x)
           
    # Save seismograms 
#    Seismogram[0,n]=txx[xrec1,zrec1]
#    Seismogram[1,n]=txx[xrec2,zrec2]
#    Seismogram[2,n]=txx[xrec3,zrec3]
    seismo[1:nx,n]=txx[1:nx,5]
    seismoy[1:nx,n]=txz[1:nx,5]
    seismotzz[1:nx,n]=tzz[1:nx,5]

In [ ]:
# Plot Seismograms 3 pts below free surface
    
fac=0.1
ymx=abs(seismo).max()
clim=np.array([-fac*ymx, fac*ymx])
plt.ion()
fig2 = plt.figure()
plt.title('Common Shot Gather (horizontal component phone)')
plt.xlabel('X (m)')
plt.ylabel('Time (s)')
im = plt.imshow(seismo.transpose(),vmin=clim.min(), vmax=clim.max(), animated=True,interpolation="nearest", cmap=plt.cm.RdBu)
fig2.colorbar(im)
plt.show()

ymx=abs(seismoy).max()
clim=np.array([-fac*ymx, fac*ymx])
plt.ion()
fig3 = plt.figure()
plt.title('Common Shot Gather (vertical component phone)')
plt.xlabel('X (m)')
plt.ylabel('Time (s)')
im = plt.imshow(seismoy.transpose(),vmin=clim.min(), vmax=clim.max(), animated=True,interpolation="nearest", cmap=plt.cm.RdBu)
fig3.colorbar(im)
plt.show()

ymx=abs(seismotzz).max()
clim=np.array([-fac*ymx, fac*ymx])
plt.ion()
fig4 = plt.figure()
plt.title('Common Shot Gather (tauzz component phone)')
plt.xlabel('X (m)')
plt.ylabel('Time (s)')
im = plt.imshow(seismotzz.transpose(),vmin=clim.min(), vmax=clim.max(), animated=True,interpolation="nearest", cmap=plt.cm.RdBu)
fig4.colorbar(im)
plt.show()

plt.ion()
fig5 = plt.figure()
plt.title('P-wave Velocity Model (m/s)')
plt.xlabel('X (m)')
plt.ylabel('Time (s)')
im = plt.imshow(model_vp,vmin=model_vp.min(), vmax=model_vp.max(), animated=True,interpolation="nearest", cmap=plt.cm.RdBu)
fig5.colorbar(im)
plt.show()